In [5]:
import os
import re
import numpy as np
import pandas as pd
import cv2
import subprocess
from functions import *
import PyPDF2
from datetime import datetime

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
main_folder = r"C:\Users\Lenovo\OneDrive\Desktop\Folders\NaharOm\BSA\Main_Project\BankStatement\All_Bank_BS"

In [6]:
excel_files = [f for f in os.listdir(main_folder) if f.endswith('.xlsx')]

excel_files_list = [os.path.join(main_folder, f) for f in excel_files]

excel_files_list

['C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\01.04.21 to 30.06.21_1.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\1229175000000237 _ 01-APR-2020_31-MAR-2021_unlocked - Copy.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\1229175000000237 _ 01-APR-2020_31-MAR-2021_unlocked.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\916020063638381-01-04-2020to31-03-2021 AXIX BANK-unlocked.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\APR2020 TO SEPT2020.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\BOB - SB.xlsx',
 'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\All_Bank_BS\\CA

In [6]:
excel_files_list = [r"C:\Users\Lenovo\OneDrive\Desktop\Folders\NaharOm\BSA\Main_Project\BankStatement\LANDCRAFT DEVELOPERS\LANDCRAFT DEVELOPERS\01.02.23 to 31.03.23_1\ExcelData\01.02.23 to 31.03.23_1.xlsx"]

In [7]:
num = 0

table_path = excel_files_list[num][:-5]+"_Standardized_Output.xlsx"
table_path

'C:\\Users\\Lenovo\\OneDrive\\Desktop\\Folders\\NaharOm\\BSA\\Main_Project\\BankStatement\\LANDCRAFT DEVELOPERS\\LANDCRAFT DEVELOPERS\\01.02.23 to 31.03.23_1\\ExcelData\\01.02.23 to 31.03.23_1_Standardized_Output.xlsx'

In [8]:
#[0,2,3,4,5,6,8,9,12,14,16]

In [9]:
excel_sheets = pd.read_excel(excel_files_list[num], sheet_name=None,header=None)

In [10]:
# drop rows with all nan values -
for (sheet,sheet_name) in zip(excel_sheets,excel_sheets.keys()):
    excel_sheets[sheet].dropna(axis=0, how='all', inplace=True)
    excel_sheets[sheet].reset_index(drop=True, inplace=True)


In [14]:
excel_sheet_list = []
for (sheet,sheet_name) in zip(excel_sheets,excel_sheets.keys()):
    if len(excel_sheets[sheet]) >= 5 and excel_sheets[sheet].shape[1] >= 4:
        excel_sheet_list.append(excel_sheets[sheet].copy())

In [15]:
terminology = None
for i in range(len(excel_sheet_list)):
    while True:
        row_1 = excel_sheet_list[i].iloc[0]
        row_1_concatenated = ' '.join(row_1.dropna().astype(str))
        words_count = len(row_1_concatenated.split())
        if words_count > 15:
            excel_sheet_list[i].drop(labels =0,inplace=True)
            excel_sheet_list[i].reset_index(drop=True,inplace=True)
            continue
        
        if "balance" in row_1_concatenated.lower() and (("description" in row_1_concatenated.lower() or "particular" in row_1_concatenated.lower() or "narration" in row_1_concatenated.lower() or "remarks" in row_1_concatenated.lower()) or ("date" in row_1_concatenated.lower())):
            if "date" not in row_1_concatenated.lower() and "date" in ' '.join(excel_sheet_list[i].iloc[1].dropna().astype(str)).lower():
                excel_sheet_list[i].iloc[0] = excel_sheet_list[i].iloc[0].astype(str) + ' ' + excel_sheet_list[i].iloc[1].astype(str)
                excel_sheet_list[i].drop(labels =1,inplace=True)
                excel_sheet_list[i].reset_index(drop=True, inplace=True)
            terminology = excel_sheet_list[i].iloc[0]
            break
        else :
            excel_sheet_list[i].drop(labels =0,inplace=True)
            excel_sheet_list[i].reset_index(drop=True,inplace=True)

    if len(excel_sheet_list[i].iloc[0].dropna()) < 3:
        terminology = None
    if terminology is not None:
        print(f"Terminology found in sheet {i+1}")
        break

Terminology found in sheet 1


In [16]:
terminology

0                  Tran Date
1                 Value Date
2    Transaction Particulars
3                     Chq No
4                Amount(INR)
5                      DR/CR
6               Balance(INR)
7                Branch Name
Name: 0, dtype: object

In [17]:
for i in range(len(excel_sheet_list)):
    diff = excel_sheet_list[i].apply(lambda x : diff_first_last_non_nan(x),axis=1)
    excel_sheet_list[i] = excel_sheet_list[i][diff>2].copy()
    
    for j, row in excel_sheet_list[i].iterrows():
        if  row.dropna().apply(lambda x: is_pure_string(x)).all():
            excel_sheet_list[i].drop(labels = j,inplace=True)
    excel_sheet_list[i].reset_index(drop=True, inplace=True)

In [18]:
for i in range(len(excel_sheet_list)):
    date_cell_prev = None
    for j, row in excel_sheet_list[i].iterrows():
        date_cell = None
        for k, cell in enumerate(row):
            if check_date(cell):
                date_cell = k
                break
        if date_cell is None:
            excel_sheet_list[i].drop(labels = j,inplace=True)
            continue
        if date_cell_prev is not None:
            if date_cell != date_cell_prev:
                excel_sheet_list[i].drop(labels = j,inplace=True)
                continue
        else :
            date_cell_prev = date_cell
                
    excel_sheet_list[i].dropna(axis=1, how='all', inplace=True)
    excel_sheet_list[i].reset_index(drop=True, inplace=True)
    excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])            

In [19]:
excel_sheet_list[0]

,0,1,2,3,4,5,6,7
0,2023-02-01 00:00:00,2023-02-01 00:00:00,UPI/P2A/303203616264/AMIT WADH/ICICI Ban/UPI,NaN,5000,CR,1595520.24,GHAZIABAD [UP]
1,2023-02-02 00:00:00,2023-02-02 00:00:00,IMPS/P2A/303311939289/SPORTSTH/YESBANKL/Landcraf,NaN,43000,CR,1638520.24,GHAZIABAD [UP]
2,2023-02-03 00:00:00,2023-02-03 00:00:00,UPI/P2A/303435238612/PRATIMA /State Ban/NA,NaN,10000,CR,1648520.24,GHAZIABAD [UP]
3,2023-02-04 00:00:00,2023-02-04 00:00:00,BY CASH DEPOSIT-BNA/CPRH09504/9406/040223/GHAZIAB,NaN,5000,CR,1653520.24,GHAZIABAD [UP]
4,2023-02-04 00:00:00,2023-02-04 00:00:00,RTGS/ESFBR52023020452992933/MERIDIAN CITY PROJECT,NaN,2260384,CR,3913904.24,RTGS HUB
5,2023-02-06 00:00:00,2023-02-06 00:00:00,UPI/P2A/340320374316/SUPRIYA J/HDFC BANK/little,NaN,25000,CR,3938904.24,GHAZIABAD [UP]
6,2023-02-07 00:00:00,2023-02-07 00:00:00,MOB/TPFT/CHOCOHOME/918020054516753,NaN,18000,CR,3956904.24,"NAVYUG MARKET, GZD UP"
7,2023-02-07 00:00:00,2023-02-07 00:00:00,INB/715168046/TIN 2.0 CBDT TAX PAYMENT/,NaN,860000,DR,3096904.24,GHAZIABAD [UP]
8,2023-02-07 00:00:00,2023-02-07 00:00:00,INB/715167662/TIN 2.0 CBDT TAX PAYMENT/,NaN,31450,DR,3065454.24,GHAZIABAD [UP]
9,2023-02-07 00:00:00,2023-02-07 00:00:00,INB/715168640/TIN 2.0 CBDT TAX PAYMENT/,NaN,28276,DR,3037178.24,GHAZIABAD [UP]


In [20]:
non_nan_column_indexes = len(terminology_preprocessing(terminology).dropna())

In [21]:
non_nan_column_indexes

8

In [40]:
def terminology_analyzer(terminology):
    values = list(terminology.astype(np.str_))
    date_idx = []
    date_num = 0
    balance_idx = None
    description_idx = []
    withdraw_idx, deposit_idx = None, None
    amount_idx = None
    description_synonyms = ["description", "particulars", "narration","remarks"]
    withdraw_synonyms = ["withdraw", "debit"]
    deposit_synonyms = ["deposit", "credit"]
    cr_dr = False
    cr_dr_idx = None
    convention = [None,None]
    for i,value in enumerate(values):
        if value is not np.nan :
            #print(value)
            if "dr" in value.lower() and "cr" in value.lower():
                cr_dr = True
                cr_dr_idx = i
                break
        
    if not cr_dr:
        for i,value in enumerate(values):
            if value is not np.nan :
                if "withdraw" in value.lower() or "debit" in value.lower():
                    withdraw_idx = i
                    
                elif "deposit" in value.lower() or "credit" in value.lower():
                    deposit_idx = i
    
        convention = [cr_dr,[withdraw_idx,deposit_idx]]
    else:
        for i,value in enumerate(values):
            if value is not np.nan :
                if "amount" in value.lower():
                    amount_idx = i
        convention = [cr_dr,[cr_dr_idx,amount_idx]]
    
    if cr_dr==False and withdraw_idx==None and deposit_idx==None:
        for i,value in enumerate(values):
            if value is not np.nan :
                if "amount" in value.lower():
                    amount_idx = i
        convention = [True,[None,amount_idx]]        
    print(values)
    for i, value in enumerate(values):
        if value is not np.nan :
            if "date" in value.lower():
                date_idx.append(i)
                date_num += 1
            elif "balance" in value.lower():
                print(value)
                balance_idx = i
            else :
                for synonym in description_synonyms:
                    if synonym in value.lower():
                        description_idx = i
                        break
    return date_idx, date_num, balance_idx, description_idx, convention

In [22]:
dates_idx,dates_num,balance_idx,description_idx,convention=terminology_analyzer(terminology)
dates_idx,dates_num,balance_idx,description_idx,convention

([0, 1], 2, 6, 2, [True, [5, 4]])

In [23]:
dates_idx,dates_num,balance_idx,description_idx,convention = column_index_processed(terminology,excel_sheet_list)

In [24]:
dates_idx,dates_num,balance_idx,description_idx,convention

([0, 1], 2, 6, 2, [True, [5, 4]])

In [25]:
for i in range(len(excel_sheet_list)):
    if excel_sheet_list[i].shape[1] < non_nan_column_indexes:
        # add a nan column jst after description column
        excel_sheet_list[i].insert(description_idx+1, 'nan', np.nan)
        excel_sheet_list[i].columns = range(excel_sheet_list[i].shape[1])

In [28]:
excel_sheet_list[1]

,0,1,2,3,4,5
0,INWARD CLG,496169,1000000,NaN,2023-03-30 00:00:00,Positive pay data not available-5lakh and above


In [26]:
final_extracted_sheets = []
for i in range(len(excel_sheet_list)):
    try:
        extracted_sheet = excel_sheet_list[i].iloc[:,[dates_idx[0],description_idx,convention[1][0],convention[1][1],balance_idx]]

        final_extracted_sheets.append(extracted_sheet)
    except IndexError:
        pass

IndexError: positional indexers are out-of-bounds

In [97]:
final_extracted_sheets[0]

,0,2,5,4,6
0,2022-04-16 00:00:00,GST @18% on Charge,DR,18,1168456.68
1,2022-04-16 00:00:00,Consolidated Charges for A/c,DR,100,1168356.68
2,2022-04-20 00:00:00,SAK/CASH WDL/SAK286015372/095/GHAZIABAD/CASH,DR,500000,668356.68
3,2022-04-20 00:00:00,INB/711828976/INTERNET TAX PAYMENT/TDS ON PROPERT,DR,159068,509288.68
4,2022-04-20 00:00:00,INB/711829409/INTERNET TAX PAYMENT/TDS ON PROPERT,DR,159068,350220.68
5,2022-04-20 00:00:00,INB/711830071/INTERNET TAX PAYMENT/TDS ON PROPERT,DR,159068,191152.68
6,2022-04-27 00:00:00,76371483-RENTAL APR 2022 CA 27-APR-22,DR,324.5,190828.18
7,2022-05-10 00:00:00,76371483-RENTAL MAY 2022 CA 10-MAY-22,DR,324.5,190503.68
8,2022-05-12 00:00:00,RTGS/HDFCR52022051267932090/LANDCRAFT DEVELOPERS,CR,500000,690503.68
9,2022-05-13 00:00:00,RTGS/SK/UTIBR52022051300354030/095/SANJAY /HDFC B,DR,342000,348503.68


In [19]:
last_sheets = extracted_sheet_processing(final_extracted_sheets,dates_idx,dates_num,balance_idx,description_idx,convention)

                            Date Description                   CR_DR  \
0  2022-04-16 00:00:00.000000000         NaN                     NaN   
1  2022-04-16 00:00:00.000000000         NaN                     NaN   
2  2022-04-20 00:00:00.000000000         NaN                     NaN   
3  2022-04-20 00:00:00.000000000         NaN                     NaN   
4  2022-04-20 00:00:00.000000000         NaN                     NaN   
5  2022-04-20 00:00:00.000000000         NaN                     NaN   
6  2022-04-27 00:00:00.000000000         NaN                     NaN   
7  2022-05-10 00:00:00.000000000         NaN                     NaN   
8  2022-05-12 00:00:00.000000000         NaN                     NaN   
9  2022-05-13 00:00:00.000000000         NaN                     NaN   
10 2022-05-13 00:00:00.000000000         NaN                     NaN   
11 2022-05-14 00:00:00.000000000         NaN                     NaN   
12 2022-05-14 00:00:00.000000000         NaN                    

TypeError: float() argument must be a string or a real number, not 'datetime.datetime'

In [229]:
# concatenate all the sheets -
final_df = pd.concat(last_sheets,ignore_index=True)

In [230]:
final_df.head(10)

,Date,Description,Debit,Credit,Balance
0,2022-10-31,POS:THE TAMILNADU C\n/TTAMBARAM\n/230410227738,1000.0,NaN,40970.09
1,2022-10-29,NEFT:SRI ADHITYA POLYFILMS PVT LTD IOBAN223021...,NaN,30000.0,41970.09
2,2022-10-28,NACH/5962831736\n/LIC OF INDIA,2042.0,NaN,11970.09
3,2022-10-28,MOBFT to: SASIKUMAR.A\n/230109624268,5000.0,NaN,14012.09
4,2022-10-27,POS:FREEDOM FILLING/CHENNAI\n/230007026249,1000.0,NaN,19012.09
5,2022-10-25,PVT./563954564\n/airtel,700.0,NaN,20012.09
6,2022-10-25,THG PUBLISHING PRIVATE L,25000.0,NaN,20712.09
7,2022-10-22,POS:THIRUARUL\n/CHENNAI\n/229519812064,500.0,NaN,45712.09
8,2022-10-22,ePAY/To: BILLDESK PAYMENT S\n/563779382/Lic\np...,1329.0,NaN,46212.09
9,2022-10-21,IMPSAB\n/229412006782\n/UBIN0907812\n/9840235724,NaN,25000.0,47541.09


In [231]:
final_df.to_excel(table_path,index=False)